In [1]:
# First we import all the necessary libraries/modules.

import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from queue import Queue
import pandas as pd
import json
%matplotlib inline

Using TensorFlow backend.


In [2]:
import pandas as pd

#Select you source file -> excel in this case and create a pandas dataframe

df_bankdata = pd.read_csv(r'C:\Atul\Data Science\IBM\Course4\Project\P12-Churn-Modelling.csv')

df_bankdata

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [3]:
                      # Data Cleansing

In [4]:
pd.isnull(df_bankdata)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
dir(df_bankdata)
df_bankdata.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [6]:
# Data Preprocessing

In [7]:
#Now that we have a good understanding of what our data looks like, we can move towards preparing it 
#to build prediction models using scikit-learn
# We should have all the fields in numerical values which is what is required for the machine learning algorithms.
# We need to convert the "Geography" and "Gender" into numerical values
# we will use the LabelEncoder function for the same.

#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
df_bankdata['Geography'] = le.fit_transform(df_bankdata['Geography'])
df_bankdata['Gender'] = le.fit_transform(df_bankdata['Gender'])

#Display the initial records
df_bankdata.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# Identify the features and target columns:
#RowNumber            int64    Not Required
#CustomerId           int64    Not Required
#Surname             object    Not Required
#CreditScore          int64    Feature
#Geography           object    Feature
#Gender              object    Feature
#Age                  int64    Feature
#Tenure               int64    Feature
#Balance            float64    Feature
#NumOfProducts        int64    Feature
#HasCrCard            int64    Feature
#IsActiveMember       int64    Feature
#EstimatedSalary    float64    Feature
#Exited               int64    Target

#Select the columns identified as features and target. OR Drop the "Not Required" and "Target" columns from the data
cols = [col for col in df_bankdata.columns if col not in ['RowNumber','CustomerId', 'Surname', 'Exited' ]]
dataset = df_bankdata[cols]
# dropping the 'RowNumber','CustomerId', 'Surname', 'Exited' columns

#assigning the 'Exited'column as target
target = df_bankdata['Exited']

dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [9]:
# Split the dataa for Training Set & Test Set creation
#import the necessary module
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(dataset, target, test_size = 0.30, random_state = 10)

In [39]:
#Select a model: We want to predict the "Exited" customers based on the data availableBuilding The Model
##Building The Model
#Lets prepare the checklist of the available data first:
#- More than 50 samples – Check
#- Are we predicting a category – Check
#- We have labeled data? ( data with clear names like opportunity amount etc.) – Check
#- Less than 100k samples – Check
#
#Based on the checklist that we prepared above and going by the machine_learning_map we can try out the below mentioned algorithms.
#
#- Naive Bayes
#- Linear SVC
#- K-Neighbours Classifier
#
#We will implement each of the models and find the performance.

In [40]:
# First we start with Machine Learning Models
##                    Naive-Bayes :- Gaussian Naive Bayes

In [41]:
# import the necessary module
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train, target_train).predict(data_test)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

Naive-Bayes accuracy :  0.7786666666666666


In [13]:
#Now we try the LinearSVC or Linear Support Vector Classification which is a subclass of the SVM (Support Vector Machine) class

In [14]:
#import the necessary modules
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=10)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

LinearSVC accuracy :  0.7933333333333333


c:\atul\software repository\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
#   K-Neighbors Classifier

In [79]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(data_train, target_train)
# predict the response
pred = neigh.predict(data_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

KNeighbors accuracy score :  0.7313333333333333


In [15]:
## Now we try some Deep Learning model

##         Keras: 

In [10]:
#              Standardize The Data
#Standardization is a way to deal with these values that lie so far apart. The scikit-learn package offers you a 
#great and quick way of getting your data standardized: import the StandardScaler module from sklearn.preprocessing 
#and you’re ready to scale your train and test data!# Import `StandardScaler` from `sklearn.preprocessing`

from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(data_train)

# Scale the train set
data_train = scaler.transform(data_train)

# Scale the test set
data_test = scaler.transform(data_test)

In [11]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(10,)))

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

W0723 19:33:37.303442 13836 deprecation_wrapper.py:119] From c:\atul\software_repository\python\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 19:33:38.967312 13836 deprecation_wrapper.py:119] From c:\atul\software_repository\python\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 19:33:39.175711 13836 deprecation_wrapper.py:119] From c:\atul\software_repository\python\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [12]:
# Model output shape
model.output_shape

# Model summary
model.summary()

# Model config
model.get_config()

# List all weight tensors 
model.get_weights()

W0723 19:33:41.348890 13836 deprecation_wrapper.py:119] From c:\atul\software_repository\python\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0723 19:33:41.349890 13836 deprecation_wrapper.py:119] From c:\atul\software_repository\python\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                132       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


[array([[ 0.12108141,  0.35229254, -0.00639623,  0.21328592, -0.05000174,
         -0.4327762 , -0.06341496, -0.46517012,  0.16402453, -0.4338236 ,
          0.18312341, -0.23247343],
        [ 0.2847861 ,  0.18152231, -0.33743852,  0.36892003, -0.21613511,
          0.35515654,  0.28152305,  0.14397669,  0.3637789 ,  0.33330727,
          0.46434873,  0.26118124],
        [-0.31282213, -0.32317626, -0.24352297,  0.4626422 ,  0.5048017 ,
         -0.09503689, -0.09610382,  0.04546946,  0.09986615,  0.35303253,
          0.21858358,  0.4513027 ],
        [ 0.3097319 , -0.35997683, -0.06733277, -0.05113801,  0.3289153 ,
         -0.41012138,  0.122419  ,  0.2301665 , -0.28375414,  0.16265815,
          0.04855782, -0.10975149],
        [ 0.5142519 ,  0.49875468,  0.01634556, -0.310727  ,  0.3309726 ,
          0.21287882, -0.42870474,  0.09871167, -0.08867803,  0.03796089,
          0.09032404,  0.46098173],
        [ 0.48830622, -0.07491297,  0.04664069, -0.21646455,  0.30128455,
      

In [13]:
#                Compile and Fit

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(data_train, target_train, epochs=20, batch_size=1, verbose=1)

W0723 19:33:51.426347 13836 deprecation_wrapper.py:119] From c:\atul\software_repository\python\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 19:33:51.721726 13836 deprecation.py:323] From c:\atul\software_repository\python\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
7000/7000 [==============================] - 12s 2ms/step - loss: 0.4440 - acc: 0.8057
Epoch 2/20
7000/7000 [==============================] - 7s 1ms/step - loss: 0.3928 - acc: 0.8303
Epoch 3/20
7000/7000 [==============================] - 9s 1ms/step - loss: 0.3605 - acc: 0.8510
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/step - loss: 0.3473 - acc: 0.8580
Epoch 5/20
7000/7000 [==============================] - 7s 1ms/step - loss: 0.3445 - acc: 0.8609
Epoch 6/20
7000/7000 [==============================] - 7s 979us/step - loss: 0.3385 - acc: 0.8591
Epoch 7/20
7000/7000 [==============================] - 6s 831us/step - loss: 0.3364 - acc: 0.8601
Epoch 8/20
7000/7000 [==============================] - 6s 921us/step - loss: 0.3338 - acc: 0.8644
Epoch 9/20
7000/7000 [==============================] - 6s 857us/step - loss: 0.3325 - acc: 0.8639 1s - l
Epoch 10/20
7000/7000 [==============================] - 6s 813us/step - loss: 0.3299 - acc: 0.8631
Epoch 11/20

In [15]:
##    Predict

In [16]:
y_pred = model.predict(data_test)

In [17]:
#quick idea of the accuracy by checking how y_pred and y_test compare
y_pred[:5]

array([[0.5220503 ],
       [0.31409854],
       [0.38060847],
       [0.0089362 ],
       [0.76008105]], dtype=float32)

In [18]:
score = model.evaluate(data_test, target_test,verbose=1)

print(score)

3000/3000 [==============================] - 0s 122us/step
[0.36847306124369306, 0.8449999998410542]


In [ ]:
## Now we increase the no of neurons and also add more hidden layers and then observe the accuracy/loss

In [19]:
model2 = Sequential()

# Add an input layer 
model2.add(Dense(12, activation='relu', input_shape=(10,)))

# Add one hidden layer 
model2.add(Dense(50, activation='relu'))  ## increased the no of neurons
model2.add(Dense(10))   ## Add 1 more hidden layer

# Add an output layer 
model2.add(Dense(1, activation='sigmoid'))

# Model output shape
model2.output_shape

# Model summary
model2.summary()

# Model config
model2.get_config()

# List all weight tensors 
model2.get_weights()

y_pred2 = model2.predict(data_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 12)                132       
_________________________________________________________________
dense_5 (Dense)              (None, 50)                650       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 1,303
Trainable params: 1,303
Non-trainable params: 0
_________________________________________________________________


In [20]:
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model2.fit(data_train, target_train, epochs=20, batch_size=1, verbose=1)


Epoch 1/20
7000/7000 [==============================] - 8s 1ms/step - loss: 0.4173 - acc: 0.8257
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/step - loss: 0.3657 - acc: 0.8473
Epoch 3/20
7000/7000 [==============================] - 7s 1ms/step - loss: 0.3536 - acc: 0.8567
Epoch 4/20
7000/7000 [==============================] - 7s 1ms/step - loss: 0.3458 - acc: 0.8600
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/step - loss: 0.3413 - acc: 0.8603
Epoch 6/20
7000/7000 [==============================] - 8s 1ms/step - loss: 0.3374 - acc: 0.8634
Epoch 7/20
7000/7000 [==============================] - 9s 1ms/step - loss: 0.3369 - acc: 0.8656
Epoch 8/20
7000/7000 [==============================] - 10s 1ms/step - loss: 0.3326 - acc: 0.8653
Epoch 9/20
7000/7000 [==============================] - 9s 1ms/step - loss: 0.3323 - acc: 0.8666
Epoch 10/20
7000/7000 [==============================] - 8s 1ms/step - loss: 0.3288 - acc: 0.8663
Epoch 11/20
7000/7000 [=====

In [21]:
score2 = model2.evaluate(data_test, target_test,verbose=1)

print(score2)

3000/3000 [==============================] - 0s 72us/step
[0.362811345299085, 0.8476666668256124]


In [30]:
##        Now we apply PCA and see the results

print (data_test.shape, data_train.shape)

(3000, 10) (7000, 2)


In [42]:
from sklearn.decomposition import PCA 
  
pca = PCA(n_components = 2) 
  
data_train_pca = pca.fit_transform(data_train) 
data_test_pca = pca.fit_transform(data_test) 
  
explained_variance = pca.explained_variance_ratio_ 

In [32]:
# Now we run the Gaussian Naive Bayes model once again

In [43]:
# import the necessary module
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train_pca, target_train).predict(data_test_pca)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

Naive-Bayes accuracy :  0.7933333333333333


In [ ]:
## So we see that after applying PCA, we have acually improved the accuracy score